In [21]:
import openai
import os
from openai import OpenAI

# Set your OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

# Define the request to GPT-4o-mini
def send_chat_request():
    try:
        # Make the API request using the new interface
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Hello, world"},
            ],
            temperature=0.7,  # Adjust creativity level
            max_tokens=50  # Adjust as needed
        )
        
        # Print the response
        print("Response from GPT-4o-mini:")
        print(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the function
if __name__ == "__main__":
    send_chat_request()

  


Response from GPT-4o-mini:
Hello! How can I assist you today?


In [20]:
import openai
import pandas as pd
import os
from openai import OpenAI

# Set your OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

def process_questions(input_file):
    try:
        # Read the input CSV file
        data = pd.read_csv(input_file)
        
        # Ensure there's a "question" column in the CSV
        if "question" not in data.columns:
            raise ValueError("The input file must have a 'question' column.")
        
        # Prepare the output list
        output_file = pd.DataFrame(columns = ["question_id" , "answer"])
        output_file['question_id'] = input_file['question_id']

        # Loop through each question
        for index, row in data.iterrows():
            question = row["question"]
            
            # Make the API request
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",  # Use the appropriate model
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant trained to answer questions succinctly."},
                        {"role": "user", "content": question},
                    ],
                    temperature=0.7,  # Adjust as needed
                    max_tokens=50  # Adjust as needed
                )
                
                # Extract the letter corresponding to the answer
                answer = response['choices'][0]['message']['content'].strip()
                # Assuming the response contains the letter (like A, B, C, etc.)
                predicted_letter = answer[0].upper()  # Take the first letter as prediction

            except Exception as api_error:
                print(f"API request failed for question '{question}': {api_error}")
                predicted_letter = "N/A"  # Placeholder for failed predictions
            
            # Append the result to predictions
            output_file['answer'].append(predicted_letter)

        # Save to the output file
        output_file.to_csv(output_file, index=False)
        print(f"Predictions saved to {output_file}.")

    except Exception as e:
        print(f"An error occurred: {e}")

test = pd.read_csv('test.csv')

print(process_questions(test))

An error occurred: argument of type 'method' is not iterable
None


In [25]:
import openai
import pandas as pd
import os

# Set your OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

def process_questions(input_file):
    # Read the input CSV file
    data = pd.read_csv(input_file)
    
    # Ensure there's a "question" column in the CSV
    if "question" not in data.columns:
        raise ValueError("The input file must have a 'question' column.")
    
    # Prepare the output DataFrame
    output_file = pd.DataFrame(columns=["question_id", "answer"])
    output_file['question_id'] = data['question_id']

    # Loop through each question
    for index, row in data.iterrows():
        question = row["question"]
        
        # Make the API request
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with the correct model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant trained to answer questions succinctly."},
                {"role": "user", "content": question},
            ],
            temperature=0.7,  # Adjust as needed
            max_tokens=50  # Adjust as needed
        )
        
        # Extract the letter corresponding to the answer
        #answer = response['choices'][0]['message']['content'].strip()
        answer = response.choices[0].message.content.strip()
        predicted_letter = answer[0].upper()  # Take the first letter as prediction
        
        # Append the result to the DataFrame
        output_file.loc[index, 'answer'] = predicted_letter

    # Save to the output file
    output_file.to_csv("output.csv", index=False)
    print("Predictions saved to output.csv.")

# Example usage
test = "test.csv"  # Path to your input CSV file
process_questions(test)



Predictions saved to output.csv.


In [8]:
sub = pd.read_csv('sample_submission.csv')
print(sub)

    question_id answer
0             1      a
1             2      a
2             3      a
3             4      a
4             5      a
5             6      a
6             7      a
7             8      a
8             9      a
9            10      a
10           11      a
11           12      a
12           13      a
13           14      a
14           15      a
15           16      a
16           17      a
17           18      a
18           19      a
19           20      a
20           21      a
21           22      a
22           23      a
23           24      a


In [6]:
df = pd.read_csv('train.csv')
print(df)
print(df.columns)
print(df['question'])

    question_id                                           question answer
0             1  [Question]: 2. In general, reliability testing...      d
1             2  [Question]: 7. Five items were placed on test,...      c
2             3  [Question]: 10.  An engineer just ran 50 compr...      a
3             4  [Question]: 6. Humans will perform tasks more ...      a
4             5  [Question]: 7.  Brand X aircraft manufacturing...      a
5             6  [Question]: 1. Compared to quality, reliabilit...      d
6             7  [Question]: 7. What is the Best time for Relia...      b
7             8  [Question]: 6. The Product Development Maturit...      d
8             9  [Question]: 2. How should a Reliability Engine...      c
9            10  [Question]: 9.  In constructing programs for e...      d
10           11  [Question]: 1. Which of the following is NOT t...      d
11           12  [Question]: 7.  Which of the following choices...      d
12           13  [Question]: 3.  10 un